In [1]:
import pandas as pd
import numpy as np
import joblib

In [12]:
import gym
env = gym.make(
    "LunarLander-v2",
    continuous = True,
    gravity = -10.0,
    enable_wind = True,
    wind_power = 5, # between 0 and 20
    turbulence_power = 1, # between 0 and 2
)

In [13]:
# TEST
# reset the environment and see the initial observation
obs = env.reset()

# Sample a random action from the entire action space
action = env.action_space.sample()

# # Take the action and get the new observation space
new_obs, reward, done, info = env.step(action)

print("new_obs", new_obs)
print("action", action)
print("reward", reward)
print("done", done)

new_obs [ 0.01230392  1.3942673   0.61917716 -0.382874   -0.0118445  -0.10255738
  0.          0.        ]
action [-0.75401485 -0.56375504]
reward -0.5781613154847889
done False


In [4]:
# Train
from stable_baselines3 import PPO
model = PPO("MlpPolicy", env, verbose=1, device='cpu')
model.learn(total_timesteps=100000)

/home/prince/anaconda3/envs/plas/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


/home/prince/anaconda3/envs/plas/lib/python3.7/site-packages/gym/core.py:318: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  "Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future."


---------------------------------
| rollout/           |          |
|    ep_len_mean     | 122      |
|    ep_rew_mean     | -331     |
| time/              |          |
|    fps             | 1674     |
|    iterations      | 1        |
|    time_elapsed    | 1        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 112         |
|    ep_rew_mean          | -313        |
| time/                   |             |
|    fps                  | 1336        |
|    iterations           | 2           |
|    time_elapsed         | 3           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.003935947 |
|    clip_fraction        | 0.0134      |
|    clip_range           | 0.2         |
|    entropy_loss         | -2.83       |
|    explained_variance   | 0.00126     |
|    learning_rate        | 0.

In [5]:
# joblib.dump(model, 'ppo.joblib')

/home/prince/anaconda3/envs/plas/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [20]:
data = dict(
observations = [],
actions = [],
next_observations = [],
rewards = [],
terminals = []
)

# gather observations
obs = env.reset()
for i in range(1000000):
    data['observations'] += [np.array(obs)]
    action, _states = model.predict(obs, deterministic=True)
    obs, reward, done, info = env.step(action)
    
    data['actions'] += [np.array(action)]
    data['next_observations'] += [np.array(obs)]
    
    data['rewards'] += [reward]
    data['terminals'] += [done]  
    
    # if i % 1 == 0:
    # env.render()
    if done:
        obs = env.reset()

env.close()

In [21]:
data['observations'] = np.array(data['observations'])
data['actions'] = np.array(data['actions'])
data['next_observations'] = np.array(data['next_observations'])
data['rewards'] = np.array(data['rewards'])
data['terminals'] = np.array(data['terminals'])

In [22]:
data['rewards'].shape

(1000000,)

In [23]:
# save data
joblib.dump(data, 'lunar_lander_data_1.joblib')

['lunar_lander_data_1.joblib']

In [ ]:
# convert to NP.array

https://www.gymlibrary.dev/environments/box2d/lunar_lander/

https://towardsdatascience.com/beginners-guide-to-custom-environments-in-openai-s-gym-989371673952